In [1]:
import random
import ipywidgets as widgets
from IPython import display
import time
import pdb

In [2]:
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two','Three','Four','Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King','Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8,'Nine':9, 'Ten':10, 'Jack':10, 
         'Queen':10, 'King':10, 'Ace':11}

In [3]:
class Card:
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
    
    def __str__(self):
        return self.rank + ' of ' + self.suit

In [4]:
class Deck:
    
    def __init__(self):
        self.deck = []  # start with an empty list
        for suit in suits:
            for rank in ranks:
                card = Card(suit, rank)
                self.deck.append(card)
    
    def __str__(self):
        deck_comp = ''
        for card in self.deck:
            deck_comp += '\n' + card.__str__()
        return 'The deck has:' + deck_comp

    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        return self.deck.pop()
    
    def decklist(self):
        return self.deck

In [5]:
class Hand:
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        self.aces = 0    # add an attribute to keep track of aces
    
    def add_card(self,card):
        self.cards.append(card)
        self.value += card.value
        if card.rank == 'Ace':
            self.aces += 1
    
    def adjust_for_ace(self):
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

In [6]:
class Chips:
    
    def __init__(self):
        self.total = 100  # This can be set to a default value or supplied by a user input
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet
        
    def blackjack_bet(self):
        self.total = int(self.total*1.5)

In [7]:
def initialize_game():
    # Creates the chips, the deck, shuffles it, creates the two players and deals two cards to each
    # Then, asks for the bet
    global deck, player, dealer, slidera, buttonc, ft
    deck = Deck()
    deck.shuffle()
    player = Hand()
    dealer = Hand()
    player.add_card(deck.deal())
    player.add_card(deck.deal())
    player.adjust_for_ace()
    dealer.add_card(deck.deal())
    dealer.add_card(deck.deal())
    dealer.adjust_for_ace()
    
    display.clear_output()
    
    if ft:
        print('Welcome to BlackJack!')
        ft = False
    print('How much do you want to bet?')
    slidera = widgets.IntSlider(value=(chips.total)/2,min=0,max=chips.total)
    buttonc = widgets.Button(description='Make Bet',button_style='info') 
    display.display(slidera)
    display.display(buttonc)
    buttonc.on_click(on_button_clicked_c)
    

In [8]:
def take_bet():
    global slidera, buttonc, bet, player, dealer
    chips.bet = slidera.value
    slidera.close()
    buttonc.close()
    display.clear_output()
    print(f'You are betting {slidera.value} chips')
    player_rounds()
    


In [9]:
def show_some(player,dealer):
    
    listplayer = []
    for i in player.cards:
        listplayer.append(i)
    print('\nPlayer has the following cards:\n')
    for i in listplayer:
        print(i)
    # find if soft or hard
    if 12 <= player.value <= 20 and player.aces == 1:
        print(f"Value = 'Soft' {player.value}")
    elif 12 <= player.value <= 20 and player.aces == 0:
        print(f"Value = 'Hard' {player.value}")
    else:
        print(f"Value = {player.value}")
    
    listdealer = []
    for i in dealer.cards:
        listdealer.append(i)
    print('\n\nDealer has the following cards:\n')
    print('Secret card')
    for i in listdealer[1::]:
        print(i)
    print('\n\n')

In [10]:
def show_all(player,dealer):
    
    listplayer = []
    for i in player.cards:
        listplayer.append(i)
    
    listdealer = []
    for i in dealer.cards:
        listdealer.append(i)
        
    print(f'\nPlayer value is {player.value} and has the following cards:\n')
    for i in listplayer:
        print(i)
    print(f'\n\nDealer value is {dealer.value} and has the following cards:\n')
    for i in listdealer:
        print(i)
    print('\n\n')

In [11]:
def player_rounds():
    global player, deck, dealer
    
    if player.value == 21 and len(player.cards) == 2 and dealer.value != 21:
        blackjack_win(player, chips)
    elif player.value < 21:
        hit_or_stand()
    elif player.value > 21:
        display.clear_output()
        player_busts(player, chips)
    elif player.value == 21 and len(player.cards) != 2:
        display.clear_output()
        dealer_rounds()
    

In [12]:
def dealer_rounds():
    global player, deck, dealer, chips, bet
    
    while dealer.value < 17 :
        hit(deck, dealer)
        
    if player.value > dealer.value:
        display.clear_output()
        player_wins(player,dealer,chips)
    elif dealer.value > player.value and dealer.value < 21:
        display.clear_output()
        dealer_wins(player, dealer, chips)
    elif dealer.value > 21:
        display.clear_output()
        dealer_busts(player, dealer, chips)
    elif player.value == dealer.value:
        display.clear_output()
        push(player, dealer)

In [13]:
def hit_or_stand():
    global buttond,buttone, player, dealer
    
    show_some(player, dealer)
    buttond = widgets.Button(description='Hit',button_style='info')
    buttone = widgets.Button(description='Stand',button_style='info')
    display.display(buttond)
    display.display(buttone)
    buttond.on_click(on_button_clicked_d)
    buttone.on_click(on_button_clicked_e)

In [14]:
def hit(deck,hand):
    
    hitcard = deck.deal()
    hand.add_card(hitcard)
    hand.adjust_for_ace()

In [15]:
def player_wins(player, dealer, chips):
    if player.value == 21:
        print(f"Player wins! Cards value is exactly 21 and Dealer's cards value is {dealer.value}")
        chips.win_bet()
        print(f'Player has now {chips.total} chips')
        ask_for_playagain()
    
    elif player.value > dealer.value and player.value < 21:
        print(f"Player wins! Player's cards value is {player.value} and Dealer's cards value is {dealer.value} ")
        chips.win_bet()
        print(f'Player has now {chips.total} chips')
        ask_for_playagain()

In [16]:
def blackjack_win(player, chips):
    print('Blackjack! Player wins')
    chips.blackjack_bet()
    ask_for_playagain()

In [17]:
def player_busts(player,chips):
    print(f"Player loses. Player's card value is {player.value}. Dealer wins")
    chips.lose_bet()
    print(f'Player has now {chips.total} chips')
    ask_for_playagain()

In [18]:
def dealer_wins(player, dealer, chips):
    if dealer.value == 21:
        print('Dealer wins. Cards value is exactly 21.')
        chips.lose_bet()
        print(f'Player has now {chips.total} chips')
        ask_for_playagain()
    
    elif dealer.value > player.value and dealer.value < 21:
        print(f"Player loses. Player's cards value is {player.value} and Dealer's cards value is {dealer.value}")
        chips.lose_bet()
        print(f'Player has now {chips.total} chips')
        ask_for_playagain()

In [19]:
def dealer_busts(player, dealer, chips):
    print(f"Player wins! Player's cards value is {player.value} and Dealer's cards value is {dealer.value}")
    chips.win_bet()
    print(f'Player has now {chips.total} chips')
    ask_for_playagain()

In [20]:
def push(player, dealer):
    print(f'Push! Player and dealer tie.{player.value}/{dealer.value} Bet is returned')
    print(f'Player has now {chips.total} chips')
    ask_for_playagain()

In [21]:
def ask_for_playagain():
    global buttonf, buttong
    
    if chips.total == 0:
        print('GAME OVER. You ran out of chips. Thanks for playing')
    else:
        print('\nPlay again?')
        buttonf = widgets.Button( description='Yes', button_style='info')
        buttong = widgets.Button( description='No', button_style='info')
        display.display(buttonf)
        display.display(buttong)
        buttonf.on_click(on_button_clicked_f)
        buttong.on_click(on_button_clicked_g)

In [22]:
def on_button_clicked_a(b):
    buttona.close()
    buttonb.close()
    initialize_game()

In [23]:
def on_button_clicked_b(b):
    print('Thanks for playing!')
    buttona.close()
    buttonb.close()

In [24]:
def on_button_clicked_c(b):
    slidera.close()
    buttonc.close()
    take_bet()

In [25]:
def on_button_clicked_d(b):
    global deck, player
    hit(deck,player)
    buttond.close()
    buttone.close()
    display.clear_output()
    player_rounds()
    

In [26]:
def on_button_clicked_e(b):
    buttond.close()
    buttone.close()
    display.clear_output()
    dealer_rounds()
    
    

In [27]:
def on_button_clicked_f(b):
    buttonf.close()
    buttong.close()
    display.clear_output()
    initialize_game()
    

In [28]:
def on_button_clicked_g(b):
    buttonf.close()
    buttong.close()
    display.clear_output()
    print(f'Thanks for playing. You leave with {chips.total} chips.')

In [29]:
def start_playing():
    global buttona, buttonb, audio
    
    print('BACKJACK 2.0  (Now with buttons!)')
    #audio
    display.display(buttona)
    display.display(buttonb) 
    buttona.on_click(on_button_clicked_a)
    buttonb.on_click(on_button_clicked_b)
    print('')
    print('')
    print('by jaume')
    print('2020')
    play_audio()

In [30]:
def play():
    global buttona, buttonb, chips, ft, audio

    buttona = widgets.Button(description = 'Start playing!', button_style = 'info')
    buttonb = widgets.Button(description = 'Quit', button_style = 'info')
    chips = Chips()
    ft = True
    start_playing()
    

In [31]:
def play_audio():
    global audio
    audio = display.Audio('8ball.wav', autoplay=True)
    print('playing audio')
    return audio

In [32]:
#play_audio()

In [36]:
play()